<a href="https://colab.research.google.com/github/pedrov718/classifying_mental_health_with_NN/blob/main/reddit_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping Posts and Comments from 8 different mental health subreddits

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
%%time

import praw
import pandas as pd
from praw.models import MoreComments


#Create an instance of reddit class
reddit = praw.Reddit(client_id="GI_ZqSXXcV12KPsNELGvAw", #my client id
                     client_secret="G5uosCAwkAEPRag1h8x01NfYosmy0g",  #your client secret
                     user_agent="Pedro_Velazquez", #user agent name
                     username = "pedrov718",     # your reddit username
                     password = "Chispit@73")     # your reddit password


# Create sub-reddit instance
subreddits = ['depression','Anxiety', 'bipolar', 'BPD', 'schizophrenia', 'autism']

posts_dict = {'top': {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Origin": [], "Post URL": []
              }, 'hot': {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Origin": [], "Post URL": []
              }, 'new': {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Origin": [], "Post URL": []
              }, 'controversial': {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Origin": [], "Post URL": []
              }}

filters = ['top', 'hot', 'new', 'controversial']

CPU times: user 398 µs, sys: 319 µs, total: 717 µs
Wall time: 738 µs


In [ ]:
for sub in subreddits:
  subreddit = reddit.subreddit(sub)

  post_filters = [subreddit.top(limit = 1000), subreddit.hot(limit = 1000), 
                  subreddit.new(limit = 1000), subreddit.controversial(limit = 1000)]

  for filter, posts in zip(filters, post_filters): 
    # Scraping the top posts of the current month
    print(filter, posts)
    for post in posts:
      # Title of each post
      posts_dict[filter]["Title"].append(post.title)

      # Text inside a post
      posts_dict[filter]["Post Text"].append(post.selftext)

      # Unique ID of each post
      posts_dict[filter]["ID"].append(post.id)

      # The score of a post
      posts_dict[filter]["Score"].append(post.score)

      # Total number of comments inside the post
      posts_dict[filter]["Total Comments"].append(post.num_comments)

      # Name of Each Subreddit for each Post
      posts_dict[filter]["Origin"].append(sub)

      # URL of each post
      posts_dict[filter]["Post URL"].append(post.url)

In [46]:
posts_dict['top']['Post Text'][:10]

['This is me. Don’t get me wrong, it’s better than don’t-leave-my-bed-for-a-week depression. I am grateful I can be an independent person. But there is something uniquely horrible about being able to go to work every day, occasionally clean up after yourself, pay your bills, generally put yourself together enough to look like a human being... but that’s it. Nothing else. No social life. No hobbies. Constantly battling your mind. And being absolutely fucking exhausted all the time.',
 '',
 'But here I am, 24 years old man and doing exactly that.',
 'I’m trapped inside. Does anyone else get that feeling? My memories from the past few years are shoddy at best. I think I’m losing it.',
 "I read a lot of posts on here of people struggling with daily, debilitating depression that plagues every aspect of their lives, making it difficult to do small things like even take a shower. I feel that I am on the opposite side of the spectrum and wanted to share that it is equally as horrible.\n\nFrom 

In [32]:
reddit_posts = {}
# saving dataframes to a dataframe dictionary

filters = ['top', 'hot', 'new', 'controversial']

for filter in filters:
  reddit_posts[filter] = pd.DataFrame.from_dict(posts_dict[filter])
    

In [40]:
for filter in filters:
  reddit_posts[filter]['text'] = reddit_posts[filter]['Title'] +  reddit_posts[filter]['Post Text']

In [43]:
posts = pd.concat([reddit_posts['top'],reddit_posts['hot'], reddit_posts['new'], reddit_posts['controversial']]).drop_duplicates().reset_index(drop=True)

In [45]:
posts.shape

(19687, 8)

In [54]:
posts.groupby(posts['Origin']).count()

,Title,Post Text,ID,Score,Total Comments,Post URL,text
Origin,,,,,,,
Anxiety,3119,3119,3119,3119,3119,3119,3119
BPD,3260,3260,3260,3260,3260,3260,3260
autism,3444,3444,3444,3444,3444,3444,3444
bipolar,3272,3272,3272,3272,3272,3272,3272
depression,3301,3301,3301,3301,3301,3301,3301
schizophrenia,3291,3291,3291,3291,3291,3291,3291


In [55]:
#posts.to_excel('mental_health_posts_scraped.xlsx')